#### 2nd Class Concurrency

- Threads, There are probably 100 different processors on the Mac and they are all running separetly and they are 3 clocks running those processors
- Concurrency is the notion that lots of things are happening at the same time
- Parallelism is doing things concurrently for the purpose of speed
- Concurrency is not parallelism(for the purpose of speed), concurrency is for functionality reasons. 
- 
Concurrency they may or may not be running at the same time but assume that they are.
#### 527 Threads
- Abstractions that are built and formulated, to manage concurrency and its **threads**
- Threads in languages like java are a language level abstraction
- Threads in c are an os abstraction. java is actually calling pthreads abstraction from the JVM and its implemented in Linux but C has access to the library called pthreads.
- What is a thread: high level, three things: 
    - sequential list of instructions that are being exeuted
    - set of variables that are local that to that seq list of instructions
    - and a set of variables that are global to the program
    - in terms of architecture, its a program counter, its a stack, and some global variables
        - PCounter says whats the next instruction
        - you have local variabvles live on an abstraciton called the stack
        - Each thread has its OWN stack,
        - multiple threads, multiple stacks, multiple program coutners
        - 1 gigantic collection of global variables that all threads have access to
- Why use threads?
    - They allow you to deal with asynchronous events synchronously and efficiently.
    - They allow you to get parallel performance on a shared-memory multiprocessor.
- 852 Process or a linux process
- A linux process Is a single thread
- Process is an address space, there should be 1 or more threads of execution
- Multiple threads live within a single process, all executing processes of linux have at least one thread
- Address space has registers, memory, access to io, everything except for the stacks are shared by all threads running in the process
#### 1152 Sync/ Async
- Discs are called syncronous devices, you ask to write to the disc, the disc interrupts with im done. syncronous because i initiate an operation and then i wait. 
- The network is not syncronous. Interrupt occurs before the request so its asynrcronous
- syncronous the request occurs before the interrupt
- **test question** what is the difference between sync/async
- sync: os issues command to device, and then an interrupt from device says done
    - console write is sync
    - irrelavant to clock
- async: device interrupts the OS saying there is something to do. and OS makes a call to do it
    - console read, network
#### 1424 Go Through C Code
- Program that takes random numbers and avgs them
    - Main takes two arguments argc, and argv 
    - the shell passes this in
    - define local variables immediately after 
    - only two types of variables 
        - local variables live on the stack and are typed
        - and global
    - atoi, ascii to interger,  takes a string and converts it to an int
``` c
	count = atoi(argv[1]);	/* count is first argument */ 
```
    - argv[0] is the name of the program itself
    - argv[1] is the first argument passed to program in shell
    - they are both strings, strings in c are linear lists of characters with a null at the end
    - char * *, argv is a pointer to a pointer to a character
    - malloc, go out to global memory space,
``` c
data = (double *)malloc(count * sizeof(double));
```
- 24 cast/ coercion
    - cast, the programmer is smarter than the compiler
    - take whatever came back without changing its representation and 
    - floating point = integer, the compiler will automatically change the int to floating point
    - when casting as long as its the same size, it doesnt change the pointer
    - Coercion do divide
``` c
printf("the average over %d random numbers on (0,1) is %f\n",
			count, sum/(double)count);        
```
---
``` c
    void *SumThread(void *arg)
    // SumThread is function that thread will execute
	pthread_t thread_id;
    data = (double *)malloc(count * sizeof(double));
	err = pthread_create(&thread_id, NULL, SumThread, (void *)args);
    err = pthread_join(thread_id,(void **)&result); //??
/*
  ??  The first argument to this call is the identifier (filled in by the call to pthread_create() when the thread was created. The second argument is an out parameter of type (void **). That is, pthread_join() takes a pointer to a (void *) so that it can return the (void *) pointer passed back from the thread on exit.
This "pointer to a pointer" parameter passing method often confuses those new to pthreads. The function pthread_join() needs a way to pass back a (void *) pointer and it can't use the return value. In C, the way that a function passes back a pointer through an out parameter is to take a pointer to that kind of pointer as a parameter. Notice that the type of result is (struct result_struct *). My using the & operator, the parameter passed is the address of result (which is a pointer) and that "pointer to a pointer" is cast as a (void **). ??
*/
    printf("the average over %d random numbers on (0,1) is %f\n",
			count, result->sum / (double)count);
    //result was malloced in SumThread
    //? should also free args
	free(result);
	free(data);
```
- ? Where is the marshalling
- The pthreads standard specifies that threads begin on function boundaries, in this example, is SumThread(). This "first" function can call other functions, but it defines the "body" of the thread. We'll call this first function the "entry point" for the thread.
- the entry point funtion take one argument that is of type (void *)
- the entry point function return a single argument that if of type (void *)
- (void *) pointer is that can legally point to any data type
- A structure define your own composite data type. In a pthreads program, the assumption define your own data type for the input parameters to a thread and also one for the return values. This way you can pass what ever arguments you like to your threads and have them return arbitrary values.
- thread's job to unpack the message into a structure that it understands. This process is called "unmarshaling" which refers to the process of translating a set of data types from a generic tranport form to one that can be processed locally. 
``` c
        my_args = (struct arg_struct *)arg;
```
- The pthread_create() call takes four arguments and returns a single result. The arguments are
    - a pointer to a variable of type pthread_t (as an out parameter)
    - a pointer to a structure indicating how to schedule the thread (NULL means use the default scheduler)
    - the name of the entry point function
    - a single pointer to the arguments as a (void *)
- main function is a running by a thread that linux "spawned" for you, main thread
- This point is important All of the threads are independent and they can run in any order once they are created. They interleave their execution with each other and the main thread (or exencute in paralle if multiple cores are available). However a call to pthread_join() ensures that the calling thread will not proceed until the thread being joined with completes.
-  Thus pthread_join()
blocks is the thread being joined with hasn't yet exited
unblocks immediately if the thread being joined with has already exited
- **?? pthread_join(), syncronization, "the state of a concurrent program is consistent across two or more concurrent  events." ??**
- **That's right -- using 10 threads only speeds it up with 0.2 seconds. Can you figure out why? **
    - comment out the printf statements to see why
    - Don't see any improvements by getting rid of the printf statements
        - reason for this is that the rand line fills up the caches, making it much faster compared to the slow filling up of memory
 
---
async vs sync
*null
malloc is void star
floating equal integer
4 = 4.0 representation will be different
coercesion int -> floating
dealing with pointers you use casts not coercions

determine seq. srand(unsigned int);. convetion: of 0. make it into the clock?
to divide double by int its a coercion
threads can only start at function boundary in C

void * is a pointer to anything

packaging the arguments is called martialling  / unpmartially
fflush insure that a print statement is set as immediately as possible, yes use fflush
context switch is a switch between threads

there is a disadvanteage of using one thread because of 140.      

what are the repercussions of not fflushing?
out_paramater is threadId
NULL = Default scheduling
pthread_create(out parameter thread id, schedule, function, argument)
you wait to join the threads
throw asserts in code

syncronishation : joining all threads together

--- 
** Reread until code and rewrite the notes to be better**
- trap is an instruction on the x86, issues a trap instruction, OS its a syscall

        

#### Race Conditions
- Operating systems must be able to protect shared state from race conditions. 
- You can run more than one thread on each cpu
- The OS multiplexes the threads on the CPU. 
- The process of pausing one thread to run another is called pre-emption, the 2nd thread prempts the first
- You need to perform a context switch to return the saved machine state of the thread
- You don't know when premption will take place, when the timer will happen
- any interleaving of instructions that preserves the sequential order of each individual thread is legal and may occur.
- Loading, subtracting and storing has to be done atomically, without the possibility of an interruption
- **test question define a raise condition**
- raise conditions exist when there is a shared variable that is being updated
- modified by multiple threads, and no raise conditions if there is 1 thread or just reading
    - possibility that a program consisting of concurrent threads, that all legal instruciton orderings don't produce the same result
- A segment of code that must be executed sequentially without the potential interleaving of other threads is called a Critical Section.
    - The instructions within a critical section are executed atomically.
- Define Mutual Exclusion: The process of ensuring that at most one thread can be executing in a critical section is often termed mutual exclusion.
- You must critical section, create region of code that is immune to interrupts, that locks the code
##### Lock
- Definition Semantics of a lock
    - a lock is initially in the state "unlocked"
    - there is a primitive that allows a thread to attempt to "lock"/acquire the lock
    - there is a primitive that allows a thread to "unlock"/drop a lock that is locked
    - any attempt to lock a lock that is already locked, blocks the thread and puts it on a list of threads waiting for the lock to be unlocked
    - when a lock is unlocked by the thread that is holding the lock, if there are threads waiting (because they tried to lock before), one is selected by the system, given the lock, and allowed to proceed.
- all critical sections have a lock, atomic code, unlock
- ** Deadlock **, crawled out the window without unlocking
- the threads that are blocked and waiting can get put on the queue practically may be slow. spin lock is a high performance. overprovisioned environment more threads means spin locks don't work well. if you are doing  34:46
``` c
    t->s[j] = 'A'+t->id;
    t->s[j] = '\0';
    printf("Thread %d: %s\n", t->id, t->s);
```
- raise condition because the malloc for the buffer happens outside the loop for creating threads, and is shared in all the thread structs 48:06
``` c
    pthread_mutex_lock(t->lock);
    for (j = 0; j < t->size-1; j++) 
    {
        t->s[j] = 'A'+t->id;
        for(k=0; k < 80000; k++);	/* delay loop */
    }
    t->s[j] = '\0';
    printf("Thread %d: %s\n", t->id, t->s);
    pthread_mutex_unlock(t->lock);
```
- entire buffer full of 'A's, 
- note print has to be before the unlock, other wise there is right execution but wrong results printed
- DEFINITION Raise conditions exist in any program where different potential interleavings produce different outputs
    - By design you can't predict the order that the threads will run in
- DEFINITION **Raise condition second definition is fixing all critical regions that need to be done atomically??** 56
    - different intervals produce different outputs that are not part of the programmers specification
``` c 
pthread_mutex_init(pthread_mutex_t *mutex, NULL);
pthread_mutex_lock(pthread_mutex_t *mutex);
pthread_mutex_unlock(pthread_mutex_t *mutex);
pthread_attr_init(&(attr[i]));
pthread_attr_setscope(&(attr[i]), PTHREAD_SCOPE_SYSTEM);
pthread_create(&(tid[i]), &(attr[i]), infloop, (void *)&(t[i]));
```
- ** pthread_attr_setscope ??**
    - different ways to control how threads are scheduled
- 104 **one might ask on the test** where is the memory that holds the value in A before and after executing
``` c
    temp = *(my_arg->src); //thread
    //... main
    int A;
    int B;
	thread_args[0].id = 0;
	thread_args[0].count = count;
	thread_args[0].src = &A;
	thread_args[0].dst = &B;     
```
- is A a global variable: NO, is A a shared varible: 
- Does this program have a raise condition. ya
- dont understand the thing though
---
#### conditions/bounded buffer
- The mutex lock: 328
    - you are locking the variable, because threads are not part of the language so the room analaogy isn't in the code
    - make the critical section be its own function to really mimick the one room
    - theres no way the compiler can enforce that the one in/one out in the code
- pthreads defines a type for the lock : pthread_mutex_t
- you need to initialize a lock, takes a pointer to a lock variable
``` c
pthread_mutex_init(&lock, NULL)
// takes a pointer to a lock variable
// 2nd argument, you can make any thread that takes a lock implicitly drop it
```
- a **lock is a shared variable** by definition shared among all threads
    - thats why you pass the address of the lock to all the threads
    - the lock variable is sitting on the stack of main, passing poitner to the variable
    - two threads can't update the lock at the same time, hand coded by assembler so it is completely atomically. 1152
    - in this case lock variable is not global but it is shared    
    - in threads you can share private variables, amongts threads, by passing pointers to them
    - they live on the stack, if maine exits, then you get a seg fault, because a thread will access memory thad doestn't exist
        - you will core dump, because you are sharing a private variable to threads thats on the stack
        - this is why you reap all your threads
`
    - you can have multiple locks for the multiple independent critical sections, or for every shared variable in your code
    - there are two ways that threads can get access to a shared variable, passing a pointer as an argument,( if you need multilpe then use a struct or array), or make a global variable lock (outside scope of main). 1747
    - ** check out the race.abc**
    
---
-  
    



- 1 to 1 coorespondence between assembler and what the machine is doing at every single clockcycle
- theoretically one assembler instruction every clock tick per cpu
-Multiprogramming : more than one thread at a time
- Basic problem of concurrency involves resources
    - Hardware: single CPU, single Dram, single I/O devices
    - Multiprogramming APIL users think they have exclusive access to shared resources
- OS has to coordinate all activity
    - Multiples users, I/O interrupts
- Multiplex in Time is the way to achieve this
    - each "cpu" needs to keep track of PC(Program Counter), SP(Stack Pointer) and registers in some state
    - what triggers switch the cpu's?
        - timer, voluntary yield where cpu will yield
        - I/o, windows 95 would crash because of this
- Using Hardware SMT: Simutanesoul multhi threading/ hyperthreading
    - higher utilization of processor resources and cycles
- Process: Operating sytem abstraction to represent what is needed to run a single program
    - A signle, sequential stream of execution in its own address space
    - **Sequential Program Execution Stream **
        - Code executed as a single sequential stream of execution
        - Includes state of CPU Registers
    - ** Protected Resoureces** 
        - Main Memory state (contents of address space)
        - I/o state (file descriptors)
    - PCB: Process Control Block
        - process state, number, counter, registers, memory limits, list of open files
    - Context Switch is switching contexts from process 0 to process 1.
- Create a process
    - Must construct a new PCB
    - Must set up a new page tables for address space
        - expensive
    - Copy I/O state
- Difference between Program ? Process ask in class
- Shared Memory communication just make an overlap of address spaces for shared 
- Modern Lightweight process has more than one thread
    - has one address space
    - you can have multiple threads executing in one process
    - every thread has the same memory space, global variables, heap
        - I/O sate, file system, network connections
    - every thread will have its own execution stack, registers but will share code, data, files kept in TCB **Thread control Block**
    
<img  src="summary_processes.png"/>

<img src="registers.png"/> | <img  src="thread_state.png"/>
------------ | -------------
<img src="lifecycle_thread.png"/> | <img src="dispatch_loop.png"/>
<img src="context_switch.png"/>


``` c
int I_array[10];
int *J_array;

J_array = (int *)malloc(10 * sizeof(int));

printf("size of I_array: %d\n",sizeof(I_array));
printf("size of J_array: %d\n",sizeof(J_array));
```
    will print two different values even though I_array ad J_array are otherwise equivalent. Can you guess what the output is? Turns out that the sizeof() operator reports the byte size that the compiler sees in the code. Thus, sizeof(I_array) is 40 but sizeof(J_array) is 8 since on a 64-bit x86 (like the ones we use) all pointer values require 8 bytes of storage.

``` c
struct return_value
{
	int i1;
	int i2;
	double d1;
};

struct return_value *MyFunction()
{
	struct return_value *r;
	r = (struct return_value *)malloc(sizeof(struct return_value));
	r->i1 = 10;
	r->i2 = 12;
	r->d1 = 3.1415;

	return(r);
}
```
    where the fields i1, i2, and d1 are the two integers and the double. Notice that in this example, I have to use malloc() to allocate the memory for return value structure. The caller would need to free this memory when it was done with the three values -- every caller. If you forget one there is a memory leak.
    
``` c
void MyFunction(int *i1, int *i2, double *d1)
{
	*i1 = 10;
	*i2 = 12;
	*d1 = 3.1415;

	return;
}
```
    Here, the void type says that there is no return value. The function uses its parameters as pointers to the data types it produces and the * operator to store the values in those memory locations. These are called out parameters since they let the function push value out of its scope.
---
``` c
void NewFunction(int *i1, int **p1)
{
	int *j;

	*i1 = 10;

	j = (int *)malloc(sizeof(int));
	*j = 12;
	*p1 = j;

	return;
}

int main(int argc, char **argv)
{
	int I;
	int *J;

	NewFunction(&I,&J);

	printf("I: %d, J: %p, *J: %d\n",I,J,*J);
	free(J);

	return(0);
}
```
    It works this way. The function NewFunction() has two out parameters: one for an integer and another for a pointer to an integer. The caller passes the address of an integer &I and the address of a pointer to an integer &J as the arguments. The function mallocs the memory and passes a pointer to that memory through the second out parameter which main must then free.

---
``` c
int main(int argc, char **argv)
{
        int i;
	int next;
	char *p;
	char *a;
	char arg1[4096];
	int found = 0;
	int len;

	for(i=1; i < argc; i++) {
		p = argv[i];
		
		if(strncmp(p,"arg1:",strlen("arg1:")) == 0) {
		
			next = i + 1;
			if(next >= argc) {
				printf("arg1 value missing\n");
				exit(1);
			}
			a = argv[next];
			
			strncpy(arg1,a,sizeof(arg1));
			
			len = strlen(a);
			if(len >= sizeof(arg1)) {
				len = sizeof(arg1) - 1;
			}
			arg1[len] = 0;
			found = 1;
		}
	}

	if(found == 1) {
		printf("arg1 found with value %s\n",
				arg1);
	} else {
		printf("no arg1 value found\n");
	}

	return(0);
}
```			
    First off, what does the program do? It looks through the argument list passed to it for the string "arg1:" and then extracts into an array on the stack the next string. Thus, if your program's api was

    ./string3 arg1: first-argument

    it parses the argument list looking for the value immediately following the keyword "arg1:".
Run it a few times:

    ./string3
    no arg1 value found

    ./string3 arg1:
    arg1 value missing

    ./string3 arg1: dog!
    arg1 found with value dog!

    ./string3 arg1: happy dog!
    arg1 found with value happy

    ./string3 arg1: happy-dog!
    arg1 found with value happy-dog!

    ./string3 arg1 dog!
    no arg1 value found

    ./string3 Arg1: dog!
    no arg1 value found
- A string is an array of characters with a 0 in the element that indicates the end of the string. Even if the array contains more elements, the first 0 to appear designates the end. You need to make sure that the last element is a 0, especially when copying strings.
- An array of characters is represented in C as a char * -- that is, a pointer to a character.
- You need to be careful when copying strings from user input to prevent the user from trying to fool you into writing off the end of the memory you have allocated to hold a copy of the string. As a rule, it is good practice to "fence" the copy of a string to be no more that the size of the target buffer.
- It is also important to remember that C doesn't know strings exist -- the are a convention. That is, the C compiler can't "know" whether an array of bytes contains a zero in its last element. Or, rather, it can't know if there is a zero at all.

![Memory](http://www.cs.ucsb.edu/~rich/class/cs170/notes/C/fig1.png)
-Dont return pointers to local variables because those live on the stack and thus get deallocated on a return

-**Notice also that foobar() need to call free() or the memory will leak when foobar() returns. Why? Because there are no other variables in the program that hold the addresses that are necessary for free() to deallocate the memory once foobar() returns.??? **

- Race Condition
    - Theoretically on machine intstrucution per one clockcycle
    - Premption: 213 
    - premption does not change the order that was executed in the thread
    - 
    - ld r1,@richs_balance		;; thread_0
    - *** pre-empt ***
    - ld r2,@richs_balance		;; thread_1
    - *** pre-empt ***
    - sub r1, 200			;; thread_0
    - *** pre-empt ***
    - sub r2, 200			;; thread_1
    - *** pre-empt ***
    - st r1,@richs_balance		;; thread_0
    - *** pre-empt ***
    - st r2,@richs_balance		;; thread_1
    - interrupting of instructions 2:19 
    - define raise condition: 
        - exist when there is a shared variable that is being updated, it is necessary
        - multiple threads
        - that all legal instruction ordering dont produce the same output
    - You must create a critical section. we are going to lock a section of code where the fetch... 224
        - synchronization primitive
    - Lock 225
        - acquire the lock
        - if you call an unlock on then its a programming erro 228
            - critical section
            - lock()
            - executes atomically
            - unlock()
        - error conditions
            - you go into the room and die, and its called a dead lock
            - also can happen if you just dont unlock when you leave
        - critical sections should be kept as small as possible
        - spin lock is 234. hardware instruction that you need in multiprocessor
        - if there are more threads than there are processors
        - usually about 100, they put you on queue
    - ascii code race.1 244
    - around the update the print also 253.. ask about the print statement
    - you cant predict the order that the threads will come in 256
    - the output of the program for the matrix multiply will always be the same
    - 300 race abc. 303. 
    - is a global variable, is a shared variable. figure out what this program and if it has a race condition.

### Condition Variables
- 1825 
- Bounded buffer problem occurs in OS all the time when you have two threads and they are running at different speeds
    - time cycle to accessing memory on disc is 1 million times slower than cpu cycle.
- you build a finite buffer, that you can run continuosly without it ever filling up
- Condition variables are a feature of a syncronization primitive called a monitor 
- synchronization issue 
    - when its empty the one that is draining needs to do stop and do nothing 
    - if you are writing to the buffer you have to be blocked when its full
    - producer/consumer parallelism
- ![Bounded Buffer](http://www.cs.ucsb.edu/~rich/class/cs170/notes/CondVar/fig1.png)
- implementing the head and tail as circular buffers
``` c 
struct order
{
	int stock_id;
	int quantity;
	int action;	/* buy or sell */
	int fulfilled;	
};

struct order_que
{
	struct order **orders;
	int size;
	int head;
	int tail;
	pthread_mutex_t lock;
};

struct market
{
	pthread_mutex_t lock;
	int *stocks;
	int count;
};

```
``` c

struct order_que *InitOrderQue(int size)
{
        struct order_que *oq;

        oq = (struct order_que *)malloc(sizeof(struct order_que));
        if(oq == NULL) {
                return(NULL);
        }
        memset(oq,0,sizeof(struct order_que));

        oq->size = size+1; /* empty condition burns a slot */
        oq->orders = (struct order **)malloc(oq->size*sizeof(struct order *));
        if(oq->orders == NULL) {
                free(oq);
                return(NULL);
        }
        memset(oq->orders,0,size*sizeof(struct order *));

        pthread_mutex_init(&oq->lock,NULL);

        return(oq);
}

``` 3701

- there are three synchronization problems in here
    - you need to keep the head and tail straight. 
        - if the queue is full stop producing. can't move head pointer
        - if the queue is empty stop consuming. can't move tail pointer
    - you need to lock the stock market not to get a raise condition on stocks
    - when a trade is exectued the specific client that made that order must be notified by the trader
    - 32:48
- code for a lock on each stock is provided
- for circular buffers you need an extra spot to tell if its full or empty when tail and head are equal
- 48, Definition Define polling you conduct some test, if the test fails then you must drop the lock and loop back around and try again
- **Test Question where are the shared variables updated**
``` c
void *ClientThread(void *arg)
{
	struct client_arg *ca = (struct client_arg *)arg;
	int i;
	int next;
	struct order *order;
	int stock_id;
	int quantity;
	int action;
	int queued;
	double now;
	

	for(i=0; i < ca->order_count; i++) {
		/*
		 * create an order for a random stock
		 */
		stock_id = (int)(RAND() * ca->max_stock_id);
		quantity = (int)(RAND() * ca->max_quantity);
		if(RAND() > 0.5) {
			action = 0; /* 0 => buy */
		} else {
			action = 1; /* 1 => sell */
		}
		order = InitOrder(stock_id,quantity,action);
		if(order == NULL) {
			fprintf(stderr,"no space for order\n");
			exit(1);
		}
		
		/*
		 * queue it for the traders
		 */
		queued = 0;
		while(queued == 0) {
			pthread_mutex_lock(&(ca->order_que->lock));
			next = (ca->order_que->head + 1) % ca->order_que->size;
			/*
			 * is the queue full?
			 */
			if(next == ca->order_que->tail) {
				pthread_mutex_unlock(&(ca->order_que->lock));
				continue;
			}
		
			/*
			 * there is space in the queue, add the order and bump
			 * the head
			 */
			if(ca->verbose == 1) {
				now = CTimer();
				printf("%10.0f client %d: ",now,ca->id);
				printf("queued stock %d, for %d, %s\n",
					order->stock_id,
					order->quantity,
					(order->action ? "SELL" : "BUY")); 
			}
			
			ca->order_que->orders[next] = order;
			ca->order_que->head = next;
			queued = 1;
			pthread_mutex_unlock(&(ca->order_que->lock));

			/*
			 * spin waiting until the order is fulfilled
			 */
			while(order->fulfilled == 0);
			
			/*
			 * done, free the order and repeat
			 */
			FreeOrder(order);
		}
	}

	return(NULL);
}
```

```c
int pthread_cond_init(pthread_cond_t *variable, pthread_cond_attr_t *attr)//: initializes a condition variable (second argument NULL says to use pthreads defaults)
int pthread_cond_wait(pthread_cond_t *variable, pthread_mutex_t *lock)//: sleep until a signal is sent to the condition variable passed in the first argument and will reacquire the lock (passed as the second variable) before the wait completes.
int pthread_cond_signal(pthread_cond_t *variable)//: signals at most one thread blocked on the condition variable passed as its first argument.

```
- 5313 mutual exclusion is critical sectino, 
    - i must exclude from threads that wish to do, so that they happen one at a time
- spin lock. this is signalling. i must block the client thread, stopping, until a traitor signals complete
- **Test Question which threads is the order-> fullfilled shared by**
    - whatever client has created the order and whichever trader has fulfilled the order
    - ?? they are the only ones that have a pointer to order??
        - technically can't a malicious client access that shared variable? 
``` c
/*
 * spin waiting until the order is fulfilled
 */
while(order->fulfilled == 0);
```
- THIS IS STILL A LOCK!
- ** Test question what is the difference between exit a pthread_exit**
    - kills only the thread, and you can even make it return but still allows a thread waiting to join to occur

- 244 : qs for office hours, checking where the head is
    - test(full)
    - lock(l)
    - update
    - some other thread get lock add unlock
- this is signaling 255 
- threads comminucate via shared vairables 
- **?? What happens if you do this without order-> fulfilled**
- If the trader finishes and signals before the client starts spinning then the client just falls through`
---
- KT join all
- kshell forks two processes

### Bounded buffer and semaphores 
---
- dont need to understand implementation kthreads,  2 
- the polling loop is a very tight loop. 4:00
- The Speed of Solutions
    - 1 spin lock 
        - 100 clients here  spinning really slows it down: check with 100 clients but small traders
    - test under clock 10:40
``` c
pthread_mutex_lock(&lock);
while(queue->full == 1) {
	pthread_mutex_unlock(&lock);
	/*
	 * whoops -- what if it goes to not full here?
	 */
	sleep();
}
```
- Raise condition because a trader could take something of the queue, and the client will go wait for something that already happened 1356
- NEED A PRIMITIVE: You need to put yourself on the queue waiting on a signal and unlock of a lock that you are holding
ATOMICALLY
    - pthread_cond_wait(identifier for variable that trader will signal, and lock in question(to drop)
    - the address of a condition variable thats type pthread_conditino_t
    - and the address of a lock of the one that was being held at the time it calls 16:06
-**TEST Question What are the arguments to Pthread_cond_wait and why?**
    - test condition, drop the lock and sleep has a raise condition. which we need an atomic primitive cond_wait.
- you do not loop back until someone signals you, you are not signaling, you are blocked, BLOCKED
- ** Good test question is if I put two locks in there and tell you to find the bug** 1922
- signal takes one argument an identifier for the queue. and you call signal typically from within the critical section that the waiter was in when it called wait?? 2124
-** ?? cond wait, puts you on a queue and drops the lock, signal i'm going to take the first one off the queue. when you continue after a pthread_cond_wait you are holding the lock but you are not guaranteed to be the first one off the queue. a client could have run before you and this is why there is a while loop to check. **
- On CSIL the Solution 2 is much faster for the slowest case, 100c and 100 t.. you just must measure
- 3435 changing the spin lock to the 2 cases... cond wait
- and now much faster for the slowest case but now slower for the 1 to 1
- Which implementation of this code is fastest: **none is fastest in all cases**
- **?? the spin lock is super fast if there is not a lot of contention??**
    - multiple threads on a cpu/cores causes you to fill your caches and that is contention/busy
- condition varaibles work: you initialize them, you wait on them and then you signal to them 3727
- when you have been signal it doesnt mean that you are the next thread to get the lock, you are free to try to acquire the lock and so check the condition again 3828
- how big a queue depends on the difference in speed between your client and trader
- **?? Definition MONITOR 4420: primitive ** abstraction of a test under lock
    - monitor is a room that has several doors, there is a lock and inside the monitor there is a series of tests and code. anything that is executed inside the monitor is executed atomically.
    - what if you call another monitor call inside a monitor
    - the reason why these didn't catch on is cause they don't nest well
--- 
- divide by 0, the cpu interrupts itself with an exception. divide by 0 is hardcoded to throw an exception
- divide by 0 transition into kernel mode
- assembly has a trap instruction
- need to differentiate between a divide by 0, or a protection error and a system call
- lots of exception, trap is a type of exception. a trap==exception vs a trap instruction.
- locks, conditionvarible, semaphores. primitives of syncronization

    
- why did csil just get twice as fast? 210, solution 1
- 210: 
- client
- if (full)
    - wait(on some variable)

- trader
- process order
- signal(v)

- pthread condition wait = you want to atomically let go of the lock and put yourself on the queue
- 217 great test question, what are the arguments to pthreadcondition wait and why?
    - the thread is blocked and stopped, until the signal is received, cpu is 
- test qs what happens if you put two locks and they are not the same bug
- signal to a wait variable where no one is waiting is simply wait
- you call the signal from the critical section 222 fill in
- after you complete the cond-wait the signal acquires the lock but you may not get it. 
- it needs to be the while loop to test 
- reaquring the lock is not atomic.
- test under lock 
--- 
- none is fastest in all situations
- if there is not much contention then spin lock is fast
- 237 - 239 write out
- Monitors
- 243, monitor has several locks 
- anything executed in the monitor is done atomically,
- monitors have these nesting problems. 

---

### [CS170 Lecture notes -- Semaphores](http://www.cs.ucsb.edu/~rich/class/cs170/notes/Semaphores/index.html)
- 4914
- we usually just use semaphores instead of everything else
- ** Test Question: you can write a semaphore package using pthreads**
``` c
typedef struct
{
        pthread_mutex_t lock;
        pthread_cond_t wait;
        int value;
} sema;

void pthread_sema_init(sema *s, int count)
{
        s->value = count;
        pthread_cond_init(&(s->wait),NULL);
        pthread_mutex_init(&(s->lock),NULL);
        return;
}

void pthread_sema_P(sema *s)
{
        pthread_mutex_lock(&(s->lock));
        s->value--;
        if(s->value < 0) {
                pthread_cond_wait(&(s->wait),&(s->lock));
        }
        pthread_mutex_unlock(&(s->lock));
        return;
}

void pthread_sema_V(sema *s)
{

        pthread_mutex_lock(&(s->lock));
        s->value++;
        if(s->value <= 0) {
                pthread_cond_signal(&(s->wait));
        }
        pthread_mutex_unlock(&(s->lock));
}
```
- test write semaphores using, pthread t
- Definition
- Increment
    - Dijkstra called this function V(); it is also called signal, unlock, leave or release.
- Decrement
    - Dijkstra called this function P(); it is also called wait, lock, enter, or get.

- a semaphore can only take on integer values
    - its initial value can be set with an initialization call
    - p(roben) 
        - block decrement = decremenet the i--, 
            - if(s->i < 0) 
            - wait()
    - v(erhogen) 
        -wake up increment, i ++,
            - if(s->i >= 0 ) 
            - signal()
- 54:42     
- ** how many threads are waiting** 
- if signal occurs before the wait you remember the increment
- v is the wake up, p is the block, and conditioned on the integer value which you get to set
- implement locks with a semaphore to set init to 1, mutexclusion
- you need to put 100 before its full, set semaphore 100. clients will decrement and traders will increment
- solves bounded buffer on the full side by setting the intial value to the number of slots
- on the trader, how many empty slots are there?
    - Set Semp to 0
    - Trader calls P on semp
    - the client will call v, signals to trader that not empty
    - ** Can you keep calling p to get to the negative **
- ** ?? good exercise cond wait and semphore whats the difference ?? **
    - you can lose signals in condwait the lock and unlock for signaling
- ** test question 3 kinds of Syncronization problems**
    - mutual exclusion, atomic counter(don't over run buffer), signaling
- ** test question why invert these is it correct?: 5136:**
    - you deadlocked, client
    - if you invert them then you make enter the mutex with something full
```c 
for(i=0; i < ca->order_count; i++) {
		stock_id = (int)(RAND() * ca->max_stock_id);
		quantity = (int)(RAND() * ca->max_quantity);
		if(RAND() > 0.5) {
			action = 0; /* 0 => buy */
		} else {
			action = 1; /* 1 => sell */
		}
		order = InitOrder(stock_id,quantity,action);
		if(order == NULL) {
			fprintf(stderr,"no space for order\n");
			exit(1);
		}
		
		P(ca->order_que->full); //can you invert these two lines
		P(ca->order_que->mutex);
		
		next = (ca->order_que->head + 1) % ca->order_que->size;
		if(ca->verbose == 1) {
			now = CTimer();
			printf("%10.0f client %d: ",now,ca->id);
			printf("queued stock %d, for %d, %s\n",
				order->stock_id,
				order->quantity,
				(order->action ? "SELL" : "BUY")); 
		}
		ca->order_que->orders[next] = order;
		ca->order_que->head = next;
		
		V(ca->order_que->empty);
		V(ca->order_que->mutex);
		P(order->fulfilled);
		
		FreeOrder(order);
	}
```
- definition semaphore:
    - You set the value of the integer when you create it, but can never access the value directly after that; you must use one of the semaphore functions to adjust it, and you cannot ask for the current value.
    - There are semaphore functions to increment or decrement the value of the integer by one.
    - Decrementing is a (possibly) blocking function. If the resulting semaphore value is negative, the calling thread or process is blocked, and cannot continue until some other thread or process increments it.
    - Incrementing the semaphore when it is negative causes one (and only one) of the threads blocked by this semaphore to become unblocked and runnable.
     - The all semaphore operations are atomic.

--- 
- mutexes and condition variables as tools of syncronization
- mutual exclusion, atomic counters, and signalling. 
- **?? Define Mutual Exclusion ?? The process of ensuring that at most one thread can be executing in a critical section is often termed mutual exclusion to distinguish it from other forms of synchronization.**

    - ? Notice also that it doesn't matter whether the trader threads are blocked or not. If all trader threads are busy and a new order arrives, the semaphore will go positive so the next P() call by a trader will immediately release it without blocking.

    Contrast this ability to "remember" that a V() call has happened so that the next P() call can proceed with pthread_mutex_lock() and pthread_mutex_unlock(). An unlock does not "store" the notion that an immediate wake up is needed if a lock happens afterwards. Put another way, lock/unlock depends on having a lock happen before an unlock (as in a critical section). Semaphores, however, can be used in cases where it is not possible to guarantee that a P() will always happen before its subsequent V().?
    
---
### Intro to KOS
- DEC MIPS R3000, x86 is little endian. if you are moving memory it should work for memcopy but safest if you do as an integer store through a pointer. 
- it Ran OS called ULTRIX, you are writing on a linux machine and its running a single process as user space process, your process. inside the process are a bunch of libraries like the kthreads library and you run your KOS. Inside a 1 megabyte long main memory.
- KOS is your bootstrap, its job is to get a userprocess running and you never doing anything with KOS again. 
- **DEFINE DEFINITION what do we mean when we say stack? : is a datastructure that is implemented by machine instructions in memory**
- You push your registers first, then push the arguments, stackreg is decremeing as you are pushing. You have to put the highest your stack can ever be and so you put that in the PCB and check
- overwriting the stacks and frames. pretend mainmemory starts 40 bytes lower
- crosscompiling, meaning the compiler is meant for R3000 assembly instructions and you can't run that on x86
- Registers 2 contains the return value because of the ULTRIX convention. first 6 registers contain arguments by convention. generally arguments are passed on the stack
- part of the bootstrap is to make a little tiny file system called the ram disk, necessary binaries to get the rest of linux running
- load user program is emulating a bootstrap so you can get your first program into memory
- kthreads gives you a blocakable system call 3316 ** listen again**
- there are only two ways the OS gets control, an interrupt and an exception
- kt fork but if you don't block your main thread, kt joinall blocks the exception thread and makes the forked threads run. once the children threads run and then die or block
- you need to be able to create a blockable function so you need to make a thread because a thread can block, functions can't block. and kt threads are not premptable
- 
![Memory KOS](http://www.cs.ucsb.edu/~rich/class/cs170/notes/IntroKOS/fig1.png)

### [Class on stuff Kthreads](http://www.cs.ucsb.edu/~rich/class/cs170/notes/Kthreads/index.html)
```c
void *kt_fork(void (*func)(void *), void *arg);
void kt_exit();
void kt_join(void *kt_id);
void kt_joinall();// The function kt_joinall() is a useful function that causes the current thread to block until all other threads have either exited or blocked on a semaphore.
void *kt_self(); //returns thread id
void kt_yield();
void kt_sleep(int sec);

kt_sem make_kt_sem(int initval);
void kill_kt_sem(kt_sem ksem);
void P_kt_sem(kt_sem ksem);
void V_kt_sem(kt_sem ksem);

```
- no speedups though because Kthreads doesn't use multiple processors -- it is strictly for decomposing a problem into thread like task
- Motherboard's job is to move data from screen, keyboard, mouse, network, graphics, atomic clock, memory to cpu and back.
- Interrupt Structure
    - CPUS have a way for the motherboard to tell them to stop what they're doing at this moment and do something else
    - board raises a signal on a pin 
    - range of interrupt codes 10 - 67 , 625
    - there is line and bus 
- the bootstrap is written and you dont see this.
- install unambigusly the data structures necessary to make sense of devices that wish to interrupt the CPU
- An interrupt does
    - cpu has general purpose registers, segment register
    - when you send an interrupt
        - CPU drops its registers into some place in memory usually to the stack
        - it is as if you make a function call by the power line
        - you need save everything as you transition because you are going to go back to doing what you were doing
- all programs that run get access to code that is written once and not like printf which the compiler attaches via library.
    - kernel is a big dynamically attachable library that has routines like
        - write to screen (puts an e on screen), came from manufacturer to the screen
        - someone has written a driver for the screen and you make an interface to the screen standard
        - these lowlevel routines are designed without access control. They are god of their device they can see everything can do. they can do everything the device cand do ... 1543
        - You have to trust
    - ** Definition The notion of calling another piece of code that is trusted is called protected transfer.** 1649
    - The process of transitioning from your program to the OS is called a protected transfer and it occurs when trap or interrupt occurs.
    - if you are in foo calling goo then that is unprotected transfer because it is assumed that if you got access to foo and goo is in the same program.
- When we tranfer from the scope of the user program to the system call we have to protect, make sure. kernel gets to test if the transfer is legal. Do this by
    - CPU has atleast one bit that is the user mode bit
    - anything that can give you access to this kernel if usermode is set will generate a fault and kill your program => death, core dumps, aborts 1853
    - protected tranfer is a special kind of function! call
    - compiler needs to know how to compile a special function! call
    - This special function name is called a system call
        - compiles it differently, have to agree on what are the supported system calls
        - includes an instruction called trap
            - trap instructurion causes the cpu to interrupt itself and changes the protection mode from user mode to kernel mode
    - trap generates an interrupt and changes the protection mode from usermode to system mode
    - these traps have intterupt codes as well and so you tell the OS to jump to this system call and only this syscall 2109
    -  Traps are generated by the CPU when your program attempts an operation that is either illegal (addressing memory out of range), impossible (divide by zero) or explicit (your program would like the OS to act on its behalf).
    - protected transfer always hits the same address depending on the system call
    - there is a table that is indexed by trap/interrupt numbers 223
    - there are 2 tables trap/interrupt numbers intialized with pointers to functions that do all of the checking for that specific interrupt in fact 2 tables and called the jump table/trap table. One for interrupts and one for traps
    - The code that the compiler generates before it calls trap. it pushes the specific syscall number, push arguments and then trapcode 2532 
    - the from jump table function call looks at the user stack all the arguments compiler pushed before calling trap
    - you call examine registers in both exception/interruption handler 32:52
        - call examine registers immediately, examine registers changes the registers, only call once
        - copy without damagin the registers the contents into a buffer
        - after you complete printf you have to come back to user program
        - need to execute at the next instruction in user code.
        - the register set has the pc where the syscall happened and the program count valuen to to jump back to 3834
        - This is slightly wrong 
        - syscall return
            - change mode back to user, do the registers and goes back to pc you specified, pc+1 3930
        - schedule process calls run usercode(register) and changes mode back goes back to location
        - slightly wrong
        - The difference between an interrupt and a trap
            - trap you go to the next instruction, interrupt you go back to the same instruction 4130
        - return syscall which is only called from trap handlers not interrupt handlers.
        - you pass back an argument from a system call, return a single value, in registers[2]
        - put all registers into the pcb
    - The intterupt handler, when the only device is the console interrupts you 5318
    - no nested switch, read interrupt or a write interrupt
    - when a machine w/ 1 process is being interrupted it could be doing one of two things
        - you can be running in userspace computing pi to 1000. you need to save user program registers
        - or you could be doing an I/O and you are waiting with nothing to run in NOOP
        - noop is the idle loop, i can only get out of it if there is an interrupt I can not get out of it if there is a trap. why because if it was a trap then there was a user program running and i so i wasnt idel 5518
    - idle is a global viariable and set it whenever you call noop, and clear it when you run user code 5734
--- 
- write out the taking out the mutex lock for the trader on the order
- and write out how the deadlock happens
- 
---
### KThreads
- Threads
- 259, tell console to print an r, and then the console interrupt when its done
- c functions can't wait, threads can wait
- not premtable, k threads run until they die or block
    - you have to let it run, exits or block on semaphore
    - to synchronize between things that are blocked and not, consolewrite semaphore
    - 2 semaphores
    - 308 
    - in exception handler only place ktjoinall()
    - 
- asdf


---
--
- 220 on stacks 
- k threads 232
- we need to make the decision either to noop or run userprocesses in the scheduler
- 245
- semaphores
- 251 test question
- implement sempahores with pthreads

---
- what would you do to see if something is a library call or a system call?
- test question which one are library which one are system calls?
- libc.a is automaticaly loaded with your c program. calling gcc will be loaded with your program.
- look at the man 2 pages what system calls are avalaible and man 3 is for libraries.
- the stdio library has fread calls read and fwrite calls write 1252
- The difference of a library call and a system call to your code should be nonexistent. Its code you didn't write.
- All processes in Linux get a unique identifier of type pid_t
- a return that comes back from a system call is not always an error code
    - getpid(): returns the process identifier
```c 
#include < sys/types.h >
#include < unistd.h >
#include < stdlib.h >
#include < stdio.h >

/*
 * call getpid() as an example system call
 */

int main(int argc, char **argv)
{
	pid_t my_id;

	my_id = getpid();
	printf("my process id is %ld\n",(long)my_id);

        return(0);
}
```

### Files
- the POSIX Five:
    - open: opens a file specified by the first argument with the "mode" specified in the second argument, and the permissions specified in the third argument. Returns an integer >= 0 when successful and negative when not.
    - close: closes the file which both tells the OS that your process will no longer access the file (useful for managing internal OS state) and also prevents the file from being accessed further in the process due to a programming error.
    - read: reads a specified number of bytes (third argument) from a file (first argument) into a buffer (second argument) at "the current offset."
    - write: writes a specified number of bytes (third argument), from a buffer (second argument) into a file (first argument) "at the current offset."
    - lseek: changes "the current offset."

- logically contiguous stream of bytes. a file is not an array. a file is nothing more logically continous stream of bytes. 
    - stream means you must access byte 1 before byte 2. 
    - logically contiguous means if there is byte 38 there is by definition 37: there are no holes 1718
- ** only open() takes a path to the file.**
- A file is persistent, when your address space is reclaimed by the OS because the machine halts the data that is in your file is preserved on disc. if you reconstiute your address space and check the file in the space disc space it is still there. Your address space memory is not like that, your os will reclaim and clean it
- this is how you get data to live beyond the lifetime of your process
- typically more disc space than memory, and this is how you can access something that won't fit into memory
- Access to a file is going to have a very particular set of semantics
    - All files live in the file system
    - the file system has a root directory and sub directories. since file system is a tree this gives every file a unique name defined by its path in the tree from the root to file itself.
    - associate a unique identifier in your program, and only in your program that is associated with the path name
    - open call returns an integer
    - ask system to open a path which is a string to do so with certain conditions
        - open for reading, writing, reading and writing, make it if doesnt exist
    - until you close the file descriptor, the file is named by the file descriptor
    - File * is not a simple integer and is implemented through the stdio library
    - threads within a process can access the FD but not other processes because processes don't share memory 2436
    
```c
strncpy(file_name,argv[1],sizeof(file_name));
file_name[sizeof(file_name)-1] = 0;
my_file_desc = open(file_name,O_CREAT | O_WRONLY, 0600);
```
- if there is any other operation besides a write done to this file throw me an error
- you can open same file twice with different or equal parameters fd 2709
- the FD catches not only what file but what operations you intend to allow for the FD
- ** ??what if you have two programs open the same file. then the two processes don't read together. it is atomic in confines of single process. not atomic between 2 processes ??**
``` c
w = write(my_file_desc,write_buffer,write_length);
// then test if w != write length
```
- **?? last thing you do is you close fd. tells the OS to release everything kept in memory to allow 3212??**
- if you run this program twice you will see the same string both times because open always opens a file at the beginning.
    - 1st file create, creates the file
    - then it writes a string to file"asdfas" closes and then exits
    - 2nd time opens the file and overwrites the file with the same string
    - you dont append, but that is a flag that you can add
- associated with a fd there is an offset. (depending on the criterion for the open) the offset gets set to 0. 
    - open a file, read 2 bytes, write a character "X" into file
    - File before: The EOF , File After ThX EOF
    - read will move fd couter by number of reads, and write will do the same. this counter is kept in the kernel.
- ** test question what are the things that linux associates with a File descriptor **
    - fd is associated with name of file(path), style of access (R/W), and an offset 3833
- **?? the 0600 is the permissions you give the file because you are creating it, for read you would set to 0??**
    - either the 0 will fail on permissions or it wont
``` c
	r = read(my_file_desc,read_buffer,sizeof(read_buffer)-1);
    while(r > 0) {
    for(i=0; i < sizeof(read_buffer); i++) {
			read_buffer[i] = 0;
		}
		r = read(my_file_desc,read_buffer,sizeof(read_buffer)-1);
    }

```
- read from the file descriptor into the read buffer a sizeof(readbuffer-1) OR LESS
- The intention when linux was invented was that everything outside of your address space look like a file. its not true you cant. All IO can be accomplished through the file interface through reads and writes.
- **?? In case of read you say maximum buffer size that you are willing to load from the file or console. your file may be much bigger than the buffer. when you get to the EOF OS will send -1. and you break out of the loop: read 4 characters and EOF? ??** 4536
- **Test question: do these two programs with different buffer size produce the same output? : the buffer size dictates in what chunks you read the file from. but it will produce the same output depending if the buffer is big or small **
- there is a lot of buffering going on to speed up the disc access when you open a file. there are buffers that are allocated to your process to do file i/o faster. Until you close the file those buffers are left allocated. If you 
- What happens if you dont close the file, if you don't close the file then all of the cached buffers will still be used and your putting your os under memory stress. 4925
- when you are actually writing to a file, you are filling up a buffer in the kernel. when the kernel gets full it schedules a big write to the disc because its more efficient. If your comp. crashes you can corrupt the file system because there was still data in the buffers that didn't get flushed out to disc. SYNCING: is called the flushing from kernel buffers to the file system (Linux: once every 30/60 seconds)
- when you close the file the OS automatically schedules all the buffers to sync to disc. Close to not corrupt files
- When you implement the exit system call, it should really close all your open fd to sync and halt. 5330
- OFFSET IS A TYPE off_t
- The slow way to manipulate the offset is through read. Special call LSeek
```c
    API
	where = lseek(my_file_desc,offset,SEEK_SET);
    // takes a FD, and explicit offset. where this offset is set, move the file pointer to there
    // seek Pos can move backwards and forwards
    // 
```
- This is much faster than read, seek set takes 1 system call for moving 1 Tb.
- If the offset goes of the end of the file:
    - if writing is allowed the file system believes that a set extends the file
    - linux will fill in the holes with 0's
    - you can allocate a terabyte on disc with lseek
- **DEFINE File Operations: OPEN CLOSE READ WRITE SEEK. POSIX file interface**
- if you open with only read, and use lseek to go past file you will get an error 1:02
- for the most part a full file system does not stop you from doing an rm operation.
    - why would you not be able to delete a file?: administrators use logging. logs are on the root file system so you can't wedge the system. 106
-**?? imagine multiplying a million by a million using lseek** 107
- lseek is o(n)
- For various historical reasons it will choose the smallest unused file descriptor.
- The console which is not a file but a device, but it is setup so that your process can treat exactly like it was a file EXCEPT it is not opened. There are devices that don't support an offset like the console or the network. you may not lseek the console. The OS is implicitly opens fd 0 1 2: to stdin, stdout, sterror. Generally stdout and sterr for the console is the same. Convention is that you read from console on 0. you tell things to user on 1. and you tell things to user/system adminstrator on 2. Redirect from shell to a file 110
```c
int main(int argc, char **argv)
{
	char buffer[4096];

	memset(buffer,0,sizeof(buffer));

	read(0,buffer,sizeof(buffer));
	write(1,buffer,strlen(buffer));
	
	return(0);

}
// ./file-fd3 
// my dog is happy!my dog is happy!
// ./file-fd3 < fd2.out
// redirect fd2.out to stdin
```


GDB
- gdb ./executable
- b executable.c:line_number
- run -a path to test execs
- n (next)
- p variable
- p *Variable
- s (Step)
- where prints the current call stack


## Fork Tuesday Feb 2nd
- cookbook is bug, 
    - write , you assume 211
    - 
- system calls 22:44
- all processes, with one exception, are made from other processes except the first process. bootstrap makes the first process. 
- origin process is called init
- new processes by cloning and is called fork
    - int which is the pid or not?
    - result of the fork call, is before the call there is 1 process and after there are two processes
    - the processes are identical with one exception being the pid: process identifier
    - PC is in the same place for both parent and child
    - both programs become runnable after the fork
    - they do not share memory
    - you differentiate immediately with an if statement on the id. child_id != 029
    - the parent gets back a non zero ID from a fork
    - the child gets back a 0 from a fork
    - else do child stuff
```c
child_id = fork();
	if(child_id != 0) {
		my_id = getpid();
		printf("pid: %d -- I just forked a child with id %d\n",
			(int)my_id,
			(int)child_id);
	} else {
		my_id = getpid();
		printf("pid: %d -- I am the child\n",my_id);
	}
```
- if child runs first then the parent will just fall through, if not then the parent will block and wait until some child (until some child exits) ?? Exits! 
```c
		wait(&child_status);
```
-  The wait() call uses this parameter as an out parameter to deliver an integer to the parent indicating the child's exit status.
- if you pthread_create(thread) the threads share memory, processes don't share memory
- EVEN A POINTER TO A is not shared because you MADE A COPY of the ENTIRE Stack. Address Space is the Boundary
- open files is shared across processes and the offset is shared
- **Test question : you have a fd, you read some, you fork, and you read /write in parent/child**
- cant specify which child to wait, a linux add on allows you to wait on a specific child with wait(pid)
- good programming says a parent reaps(waits for the death of) all of its children
- ** when an interior node of the process tree exit the children of that node are promoted to INIT ** 4238??
- Process tree
- When you login you type a pw, that pw is translated into an int which is stored in a PCB of your login process. You get one process called the login process and its running the shell. When you type ls in shell it does: fork. For you root is the login node.
- forkbomb, just for loop a fork statement. if you aren't waiting for you children they are sort of like orphans but you could be leaking processes just like leaking memory if you don't have quotas. 
- ** DEFINE PCB : has children, parents processes, it has address space start, and size of memory, and registers, file descriptor table **
- fork a shell for rich from init, and then you log out. init will inherit. init waits for everybody. The processes left from the rich login will run in the background
- init forks a process called a getty which interegates you for the password 50
- you can use a kill command if the user id is the same to kill background processes 5139
- Linux has 150ish processes running from init/system d
- pcb add a datastrcutres called my children which will point to pcbs 103
- You also need a back pointer for the parent stored in PCB

# Exec
- Daniel / On each page put page x/y
- Bring Paper

- a fork is a complete a replicate
- exec 1519
- In linux one program can run another program. With fork one program runs itself twice. You can write a process that runs the program ls but its doesn't call it like a function
- exec kills you and then replaces you with what you want to run
- execve(/bin/ls, Null, Null)
    - 1st program makes an address space with fork
    - calls exec which fills with new code and run from the beginning
- it uses the same resources of the original process like the process id, the file descriptors, the console is still there 19
- why go through all the business of making a copy?  
    - because if you didn't then you would have two types of fork. fork and im going to exec or fork and i wont exec
- fork makes a new vessel where code can exist, calling exec fills the vessel up with new code
```c
// fork3
err = execve(file_name,&(argv[1]),envp);
/*
 * not reached if execve is successful
 */
printf("pid: %d -- execve of %s failed with error %d\n",// /bin/ls is file_name. where in the file system to find the binary
// argv[1] is pointer to /bin/ls
// call ./fork-3 /bin/ls /tmp
// argv[0]: fork_3
// argv[1]: /bin/ls
// argv[2]: /tmp
```
- DEFINE The first argument to execve() really needs to be a path to the file in the file system that contains an executable program. Thus, it will need to contain something like /bin/hostname or ./fork-1 -- a complete path name to the binary.
Next, execve() takes a list of arguments that it will pass (as the char *argv[] parameter) to the main() function of the program that is being run. In this example, the first argument passed to fork-3 will be the path name to the binary to execute and the remaining arguments that need to be pass to it to do its job.

- execve can fail if either the permissions are wrong or the file doesnt exist
- if you run ./fork3 //bin/bash: it it creates a new process with bash as a promt
- **Test question what happens if you fork bash, how many times do you have to call exit ?? 35-37**
- you start with a shell running bash, reading from stdin. you fork and exec /bin/ls. the child process inherits the FD's so stdin and stdout. It runs ls and writes to stdout 39
- the other processes of bash are waiting and that is the reason why you dont see duplicates in all the shells 41
- **?? why are there two processes started when running bash ??**
     - bash increments 2
- environment pointer 4330. all string initiated by the shell that all the children can inherit if a pointer to envp is passed to them. type in export to see all of them
- How does the child tell anything to the parent. In linux the child tells the parent one integer which is the exit code
- Anything else linux expects is going to get written into a file and the parent needs to know what file ahead of time
- Status Code is the one integer that you pass back as a child to the parent 4526
- any process that ends successfuly passes back 0. Any process thats being executed by the shell passes back a positve integer for an error. The system calls in C is success is usually 0 and negative numbers when something is wrong.
- Exit(1) returns 256 because it means something else 4731
- syswait.h contains the macros WEXITSTATUS
```c 
if(WIFEXITED(child_status)) {
    printf("pid: %d -- %s has completed with status: %d\n",
        (int)my_id,
        file_name,
        WEXITSTATUS(child_status));
}// now returns 1
```

- **Test Question what if i exec rich.txt not a compiled binary. you should try that.**
    - if file doesnt exist then error -1. also if it isn't a binary -1
- a reap is either a parent/init capture the exit status code of the process through a wait 
- what if the parent forks, computes pi, and never calls wait. the child dies, and the parent outlives the child. Linux figures it is best to store this.
- linux stores a zombie a child who has died for which the parent has not called wait
- zombies will not get cleaned unless the parent calls wait or dies. If parent dies then zombie is adopted by init
- the job of init is just to call wait() so that orphaned children can report their exit status and avoid becoming zombies, or cease to be zombies.
- zombies are problematic because they take up space in the kernel. They only take up one integer... 5352
- There is no way as super user to remove a zombie. I can kill a process but if the parent doesn't entirely die, the process isn't fully dead. 
- a zombie retains its process id 5612
- process address space can get smaller  through an exec call but it doesn't so much make the address space small but the code that contains it. what's different is the amount of you use on the stack and on the top. 5910
- fork fails if there is not enough address space on your machine to create a child of your size
- For our lab you can at most run 8 processes. get negative value for a 9th fork
- ** you aren't going to run into a problem where exec will fail because not enough size**
- all address spaces are the same size. 
- Linux does not use fixed partitions, vs virtual memory
- for next assignment we are implement mft - multiple fixed termination`
- linux process model
- 

# Pipes
- ** put your name, page x/y and problem number and BRING PAPER**
- Bring Pencils

- Any process that dies, its immdediate children become children of init 8
- fork copies it does not share.
- exec can use the file descriptors that were open 
- Pipes are used for interprocess communication
- the pipe() system call an array of two integers that it uses as an out parameter.
    - it allocates two file descriptors
    - one for reading 0
    - one for writing 1
- you can't call lseek on them and you don't call open but pipe() on them
-  First, while the write end "knows" how much data it will write, the read end doesn't. In this example, I made the read buffer 4096 bytes in length. If the writer had written more than 4096 bytes, the reader would have only read the first 4096 assuming that the pipe is big enough to hold 4096 bytes. The semantics are that the pipe has some capacity (which is not known) and if the writing program exceeds that capacity it blocks under a reader has drained the pipe there by freeing some capacity. In other words, a pipe is the Linux system call implementation of the bounded buffer problem we discussed in the lecture on condition variables. Notice that in this example, the process could "deadlock" itself by writing more into the pipe than the pipe can hold.
- Notice that the code creates a pipe before it calls fork(). 
-pipie 3 c 
- cat file.txt | grep "cs 170" 
    - cat is reading file.txt from stdin
    - the pipe | tells to connect the stdout of cat with stdin of grep
    - grep reads it and outputs all examples of cs170
    - chain programs together
- pipe takes an array of two out integers as outpointers
    - input file descriptor
    - output file descriptor
    - pair of fd, 0 is the read end, 1 is the write end
``` c
API
int pd[2]
pipe(pd)
```
- ** One process makes a pipe, writes some data into the pipe, reads some data from the pipe (bounded buffer) great test question 1943**
- ** test question if you change the read/write order what happens: DEAD LOCKS because bounded buffer and nothing to read**
- You CAN STILL DeadLOCK if you overwrite the buffer, and then the read never happens
- usually don't use a pipe to talk to yourself
- pipe is really good at talking with your children
    - you should close the end of the pipe you aren't using
- cat file.txt | grep 
    - grep is a child and the cat is a parent
    - cat does not need to read the read end
    - grep doesn not need to write the write end
    - ** the pipe is created before the fork**
- .
- parent create pipe
- parent call fork
- parent close read end
- parent writes into write end
- child close write end
- child reads from read end
- .
- it looks like a file, but there is no directory, but there is no permissions, not file... 28
- Since all of them need to share the file descriptors they all have a common ancestor. and most likely you just fork all of them 29
- .
- when you write some data into the pipe on the write end. the kernel will deliver the data even after you die.
- cat is fast, grep is slow. cat finishes writing and exits. YOU CANT KILL THE PIPE
- That means there is a difference between reading a pipe which has no data or reading a pipe which has no writer. 31
- Kernel sees everything. if client dies and there is data in the pipe. It will deliver to trader and give EOF.
- the pipe will be removed from memory once all readers and writers died 
- kernel has a reference count with readers and writers to a pipe which it increments
- this is why you should close the read end or write end.
- if you write into a pipe and there are no living readers then you get an error saying there is no reader
- There are two ways that the writer can give EOF. It Can close the pipe or exit. if it exits the kernel will close the pipe and give EOF. 35
- pipe-3c has no raise condition
- an eof is put into the pipe stream at the end of the data that will be delivered when all of the processes that hold the write end of the file descriptor close the pipe descriptor 38
- .
- all readers have to read EOF, its actually not an EOF in the buffer 

- multiple writers writing to the pipe is a raise condition
- kernel will schedule the writers and the readers in any order
- you must deliver all of the data to some reader
- if 1 2 3 is written 1 2 3 comes out to the readers
- .
- if you write 10 it is a convenience that the they are put into the buffer continuously
    - you have to test what linux does with pipes
- ** BAD CASE you have one writer and one reader **
- writer writes 100 bytes into the pipe
- bytes_read = read(pd[0], buffer, 250 bytes)
    - the reader returns 100
    - how do i know that the writer is just a little bit slower and there is a 101 coming
    - os swaps the writer and the reader
- if the buffer is not empty, there is data in the buffer and the reader has enough space to hold all of the data
- linux semantics are you do a short read, because you get back 100 out of 250 and you unblock the reader
    - if you call read on disc it will try to read that much
    - on pipes it will not 
- they failed to make read look the same from a file, a console or a pipe 4543

- cat which reads from stdin and writes to stdout until someone control d's the console
- 
- 48 another thing that promts a short read is a carriage return
- because carriage return should deliver whatever is in the buffer to user space if you want linux semantics 49
- because i have not typed 4096 bytes nothing happens without implementing the carriage return
- carriage return = moves cursor to the leftmost position and a new line character 5020
- 
- cat needs to lose the ability to write to console
- grep needs to lose the ability to read to console
- dupe says create a copy of this filedescriptor in the lowest number that is not used
```c
int fd = open("./my-file.txt",O_RDWR);
close(0);
dup(fd);
//after these comments a read, reads from stdin connected to that file
```
- cat | grep < file.txt
- shell does pipe
- shell forks itself
- shell closes the console
- shell calls dupe
- shell cals exec on cat
- a parent is going to create a chain of these things
- shell starts with stdin and stdout
- foo bar poop
- sets stdin to fo 
- sets stdout to poop if its last command
- pipes in between all others
- kos the pipe memory is wherever you put it, pipe data structure with a bounded buffer and size
- linux uses the top 1/3 of the filesystem for buffer stuff 59  

- after midterm
class on dining problem
---
## Memory Management
- qs Notice that a partition is an artificial boundary that is determined by the values of the base and limit registers chosen by the OS. For example, if there are only six different values for the base register that the OS will ever choose, the memory will have seven partitions.
- start 640
- the loader, (ld is not this program, actually linking) takes the output of the compiler x86 instructions.
- nm will find all strings in your program
    - int A
    - main
    - char *c = "rcich is mean"; //also global
- instructions are loaded at 0, next page boundary
- data segment are well gloval variables and string constants

- when you do seperate compilation the compiler makes one text segment and one data segment. 
    - thats the process of linking, taking a bunch of separate things and lining them up into one data segment is linking 
- QS when you do seperate compilation its not linking
- 
- stack and heap
    - morally the same thing
    - memory regions that compiler believes are avalaible to it at runtime
    - names may be known at compile time
- they are local variables, the stack pointer is decremented necessary to hold the local variables
- stack frame, when you enter a new a scope you push a new frame by decrementing the stack pointer to have enoug hspace for local variables 218
- there is no scope outside of main\
- the arguments that you call a function with, also get pushed onto the stack usually below the local variables
- data segment is not relocatable
- no such thing as a recursive global call because global variables do not move 223
- a.out format is
- 2 diff stack and heap, heap grows from low to high, stack grows from high memory  to low memory 224
- calling function, function preamble, when you return the stack pointer is incremented postamble, changes the stackpointer
- you maniuplate the heap through malloc
- it does this at the start of the data segment that is not part of the initialized data segment
- printf is calling malloc, sbrk me some bytes. doesnt want to take up all memory
- qs at 27.42 with the matrix 1mil * 1 mil
- 
- 228  malloc asks for abig buffer, you can overwrite variables in your malloc by offsetting pointers
- 3117 write out the example of how you overwrite malloc space
- 234 base and limit registers are blindingly fast and thats what you build a super comp. with
    - not even a full instruction to test if address is inside limit
- you need to give it an initial stack pointer, the data segment is taken care by the base register
- we are going to make all the addresses relative to the base register
- virtual memory is the idea that everybody has an address space that begins at 0
- compiler compiles them as if they are the only program in the world and its taken care of before you run the program you tell the machine the hardware everytime there is an address add the base offset and go to that physical memory
- phys mem
- mft 239 multiprocessing fixed tasks
    - your memory is broken up into fixed partitions and you can only simutanesouly run as many programs as there are partitions
    - you got  text segment, data segment, the registers, the heap, the stack and write them out to disc
- I can run another program there and bring the first one back from disc
- swapping, you can run more programs then you have segments for by swapping out to disk, 
- because of the base register it is all relocatable, change the base register and you are all good
    - really bad performance hit for swapping
- there are no physical addresses in your program its all virtual
- the base/limit implements memory protection, processes will fault if they go outside of limit, you can't have a negative memory address won't work either
- 245 fragmentation
- comes up on disc and in memory, fixed partitions are susceptible to internal fragmentation, fixed allocation regions, if the text segment and data segment is small and there are no local variables then all the memory between the data - heap - stack is wasted. 
- there are unused pieces of memory that could not be used internal to each of the allocated regions
- whats the memory utilization? you would add up the text, data and used stack pointer for all 8 divide by all the memory.
- test question, calculate the internal fragmentation, memory used/(memory used+memory allocated) 4945
- now there are no fixed partitions mvt variable tasks now the partition size doesn't dictate largest program you can run 252
- this is still relocatable, you define a variable number of partitions and those boundaries can move
- this is suspectible to external fragmentation, and this is how malloc works
- external fragmentation occurs when we allocate objects not in fixed units but in variable length units and the fragmentation is outside of the unit we are allocating 54
- 3 ways to make a placement decision
    - first fit you start at one end and you find the first hole that will fit the process 
    - best fit, take the space that is closest to the size of the process you are trying to fit
    - worst fit, sweep through the memory and find the hole that is biggest that you have
- if you have two holes next to each other you coallesce them into the larger hole
- base and limit is really virtual memory, the base register is set by the OS to a physical memory location and then it is completely obscured from the program until traps back to os. 

- 4 things wrong about lab
    - 1 ksh is not a debugging tool, its an acceptance test
    - 2 cook book in lab 2, says syscall return 
        registers[pcreg] = registers[nextpcreg]
    - next pcreg should also be set 4 ahead. 
    - 3 interrupt handler
    - 4 sbrk 5 lines of code, get your shit together
- 212
- pcreg
- midterm stuff until 240
- ?? semaphore if statement ??

## Demand Paging 
- base and limit is a very efficient, in the cpu, in the memory pipeline gets an add of Base. virtual memory, super comp use this
- number of bits in the offset defines how big apage is. offsett is amount into a single page
- **Address page**
- all address pages begin at 0, its virtual memory. BUT the machine doesn't think of it as one contiguous array but contiguous blocks of pages defined by the page size. 
- typically you break Phys Mem up into the same sized regions
    - these are called frames
- talk about pages its address space, talk about page sized frames its phys mem
- Data Structure Page Table. Create a Map
- its a one way map
- Page table indexed by page number and contains a frame number
    - implemented by the hardware
- the frames don't have to be in order, they can be seperate.
- start executing at 0, fetching from 0. 0 is on page 0. go to page table. where is the frame? frame is 2. frame is 2 and we are going to take offset which is 0. and fetch from phys mem.
- If i tell you 54.
- this is also a div and mod operation.
    - page size is 512. the offset is the address mod 512. page number is addres div 512.
- div is expensive but this is fast because always a power of two. and then its just a div and mask.
    - Some assembly have one instuction to do a shift and mask
- there is a page table base register points to the table which is a table that is malloced in the kernel. 
--- 
- Address space is logical. continuios where page 0 is before 1 and there are no gaps in between
- PCB points to the page table
- ** Virtual memory ** user base and user limit is virtual limit because every process starts at 0 at their logic eventhough they start a different phys memory. the address the process sees is virtual.
- To get **Demand Paging** we add 
    - physical frame #, valid, modified, referenced, protection bits(2 bits)
    - the hardware defines
- Acces 1536, its on page 3. goes to page table and doesn't find a valid frame #. When you create the page table entry is not set. 
- if the valid bit is clear you take an exception. the hardware triggers a fault, different than a regular exception. 
- it is triggered by the user space program but unlike system call you do not return to the address immeditately after. but here you don't bump the pc forward, you leave it where it is. - its called a page fault, because the hardware has deteceted that the processor is accessing a logical address that has is not yet currently mapped. Valid bit is clear
- Fault occurs,
- each process has its own page table. but processes can share memory if they point to the same frame number.
    - Notice also that if they were to share memory, they don't need it to have the same address in their respective memory spaces.
- Protection
    - none: no access permitted
    - read only
    - write only
    - read/write
- If these proteciton bits are violated then a protection fault is thrown.
- Read only bit will be set for all pages in the text segment of a Unix program. so you can't write to those
- Red page:  a no access page may be put between the stack and heap to allow the process to fail gracefully if you over-write either
- Data Structure: Disk Block Descriptor Talbe
    - ?




---
- New class
- one way map 206, logically lives in the pcb, it is an array
- speed reasons, size of page is power of 2
- page mapped memory
- demand paging 210
- user base and userlimit is virtual memory ...
- everybody gets 0 and up to hardware to map it
- each cpu has a format for page table 
- valid bit, modified bit, may or may not have referenced, may may or not 2 bits for protection
- these are hardware bit, os doesnt clear this every acces
- 1536, doesnt' find valid entry, when you create 214
- if the valid bit is clear you trigger an exception and its a fault. page fault
- you don't return to address immediately after the exception
- you accessing something that is not yet currently mapped 
- swap space 219 swap map
- it faults them in 223
- page faults are slow, do this
- parsimonius
- 226 start
- swap table doesnt have to be array
- page table must be array
- syncronization, you must sure its entirely in physical memory, 231
- optimization, is text segment is read only
- any mutable pages 234
- how do i find the frame 237? frame table
    - has bits that say used or not
- deallocation when process goes away. clean up your swap space. 
    - clear used bits frame table
- 239, 
- one big raise condition
- modified and referenced  and clock algorithm
- you create a page stealer, wakes up every 50 ms, 
- gets cleared only when the good copy gets flushed to disk
- programs exibit logically locality, runset
- 249
- runset,
- clean is 1, 0
- dirty page gets cleaned by pushing out dosick 
- swapping
- keeps track, doing this from perspective of 3rd tbale
- thrashing, you immediately fault a page that was stolen
- looks in wider segment
- page stealer will be shorter than 50ms to 25 ms, 
- pick a process and kick, put end of run queue, kick of all the frames, clean all of its stuff
- this swapping, lowering its, linux chooses a process with a very large memory size to swap
- 3rd one, oom, out of memory killer 
- it will start killing processes, because it doesnt know what its killing
--- 
March 1
- memory management
- 00 is protection fault 205. Red page
- crash the program immediately
- a.out format. text data are createdby compiler and heap and stack created at runtime.
- 01 is read only. 
- logically are segments but are implemented using pages
- 215 memoery
- 1 per process
- break address space into p0 space and p1 space
- hierachical page table used by x86
- divide address space into 3 regions 220
- inverted page tables used by
- test qs you have 2k page size, 64 bit address size. how big does the page table have to be..
- 2 memory refences for every one or hierachical. you can do 4
- TLB: translation look aside buffer 
- shootdown stop the translation i got it
- context switching is expensive all of your TLB cache must be discarded 231
- file system
- discs are not memory
- memory is Random access
- discs are not byte addressable
- sectors some power of 2. 512
- tracks, and some number of sector per track
- entire sector is read
- i node 244
- table of disc block numbers
- test qinternal fragmentation or external
- 304
- can you hide a directory so that it doesnt have a parent
- name i
--- 
- 2nd to last class
- 203 , large enough to hold a. 4 bytes is a block number. 
- everything besides the name of the file is stored in the inode
- access time, creation time
- 

## Last Class
- 9 pages, bring paper and pencil again
- pipes for sure
- 6 min file: 
- a file is an inode, a process is a pcb
- file descriptor has an offset
- pcb has fd table, and open file table inside kernel ref count w/ offset
- "incore"/in memory inode table with ref count and name filename
- the reason the openfile table exhists is cause of fork
- inode table is for speed 212
- open file table is for offset
- performance 220
- 227 3 least recently used get rid of it 
- 236 name_i routing from path to inode
- hard links vs soft links: many paths to same file
- you can go past mount table 247
- there is a reference count
    - so it doesnt actually remove the file?
- 254 summary
- ?? can you only mount the root?
- testqs to access root inode, look up data that is in the inode 301